In [1]:
# Import libraries

# API functions
import requests
import pandas as pd
import datetime

In [2]:
# Define Functions

def get_start_date():
  """Calculates the date for 6 months ago from the current date."""
  today = datetime.date.today()
  return today - datetime.timedelta(days = 30 * 6)  # 30 days in a month

def get_end_date():
  """Returns today's date."""
  return datetime.date.today()

In [3]:
# Calculate Dates
start_date = get_start_date()  # Or: start_date = datetime.date.today() - datetime.timedelta(days = 30 * 6)
end_date = get_end_date()  # Or: end_date = datetime.date.today()

print(start_date)
print(end_date)

2023-09-15
2024-03-13


In [4]:
# Format Dates
formatted_start_date = start_date.strftime("%Y-%m-%dT00:00:00+00:00")
formatted_end_date = end_date.strftime("%Y-%m-%dT23:59:59+00:00")

print(formatted_start_date)
print(formatted_end_date)

2023-09-15T00:00:00+00:00
2024-03-13T23:59:59+00:00


In [ ]:
# Build API URL
base_url = "https://api.esios.ree.es/archives"
date_type = "datos"
url = f"{base_url}?start_date={formatted_start_date}&end_date={formatted_end_date}&date_type={date_type}"
